In [1]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
# from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
dataset_names = {
    "adult": "Adult",
    "huga": "HuGaDB",
    "kdd": "KDD99",
    "spambase": "Spambase",
}

In [3]:

results = []

for model in ("fedlr", "fedfor"):

    for n_clients in (10, 20, 30, 40, 50, 60, 70, 80, 90, 100):

        dfs = {}

        for dataset in dataset_names.keys():

            dft = pd.read_csv(
                f"history/{n_clients}_clients_{dataset}_{model}.csv"
            )

            df = pd.DataFrame()
        
            # get percentiles (0, 10, 25, 50, 75, 90, 100)
            df = dft[[f'Client {i+1} Accuracy' for i in range(n_clients)]].T.describe(
                percentiles=[0.1, 0.25, 0.5, 0.75, 0.9]
                ).loc[['min', '10%', '25%', '50%', '75%', '90%', 'max']].T
            
            df['n_clients'] = n_clients

            df['global'] = dft['Global Accuracy']

            dfs[dataset] = df

        for comm_cost in (0.0, 0.0125, 0.025, 0.05, 0.1):
            for target_dataset in dataset_names.keys():

                X_train = pd.DataFrame()
                y_train = pd.Series(dtype=int)

                for dataset in dataset_names.keys():

                    dfs[dataset]['ground_truth'] = dfs[dataset].apply(
                        lambda row: 1*(row['global'] > row['max'] + comm_cost), axis=1
                    )
                    if dataset != target_dataset:
                        X_train = pd.concat([X_train, dfs[dataset].drop(columns=['ground_truth', 'global'])], ignore_index=True)
                        y_train = pd.concat([y_train, dfs[dataset]['ground_truth']], ignore_index=True)

                    else:
                        X_test = dfs[dataset].drop(columns=['ground_truth', 'global'])
                        y_test = dfs[dataset]['ground_truth']

                if len(y_train.unique()) < 2:
                    if y_train.unique()[0] == 0:
                        y_pred = np.zeros_like(y_test)
                    else:
                        y_pred = np.ones_like(y_test)
                else:
                    # est = LogisticRegression(max_iter=1000, random_state=42)
                    est = RandomForestClassifier(n_estimators=100, random_state=42)

                    est.fit(X_train, y_train)
                    y_pred = est.predict(X_test)

                results.append({
                    "model": model,
                    "n_clients": n_clients,
                    "comm_cost": comm_cost,
                    "target_dataset": target_dataset,
                    "accuracy": accuracy_score(y_test, y_pred),
                    "precision": precision_score(y_test, y_pred, zero_division=0),
                    "recall": recall_score(y_test, y_pred, zero_division=0),
                    "f1_score": f1_score(y_test, y_pred, zero_division=0),
                    "frequency_true": np.mean(y_test),
                    "frequency_pred": np.mean(y_pred),
                })

                print(f"Completed: model={model}, n_clients={n_clients}, comm_cost={comm_cost}, target_dataset={target_dataset}")
                print(f"  Accuracy: {results[-1]['accuracy']:.4f}, Precision: {results[-1]['precision']:.4f}, Recall: {results[-1]['recall']:.4f}, F1-score: {results[-1]['f1_score']:.4f}")
                print(f"Frequency True: {results[-1]['frequency_true']:.4f}, Frequency Pred: {results[-1]['frequency_pred']:.4f}")
                results_df = pd.DataFrame(results)
                results_df.to_csv("final/lodo_estimator_results.csv", index=False)

                



            

Completed: model=fedlr, n_clients=10, comm_cost=0.0, target_dataset=adult
  Accuracy: 0.0150, Precision: 0.0150, Recall: 1.0000, F1-score: 0.0296
Frequency True: 0.0150, Frequency Pred: 1.0000
Completed: model=fedlr, n_clients=10, comm_cost=0.0, target_dataset=huga
  Accuracy: 0.1400, Precision: 0.0000, Recall: 0.0000, F1-score: 0.0000
Frequency True: 0.8600, Frequency Pred: 0.0000
Completed: model=fedlr, n_clients=10, comm_cost=0.0, target_dataset=kdd
  Accuracy: 0.0050, Precision: 0.0050, Recall: 1.0000, F1-score: 0.0100
Frequency True: 0.0050, Frequency Pred: 1.0000
Completed: model=fedlr, n_clients=10, comm_cost=0.0, target_dataset=spambase
  Accuracy: 0.0050, Precision: 0.0000, Recall: 0.0000, F1-score: 0.0000
Frequency True: 0.9950, Frequency Pred: 0.0000
Completed: model=fedlr, n_clients=10, comm_cost=0.0125, target_dataset=adult
  Accuracy: 0.0000, Precision: 0.0000, Recall: 0.0000, F1-score: 0.0000
Frequency True: 0.0000, Frequency Pred: 1.0000
Completed: model=fedlr, n_client

In [4]:
results_df[['accuracy', 'precision', 'recall', 'f1_score', 'frequency_true', 'frequency_pred']].mean()

accuracy          0.638700
precision         0.123473
recall            0.135553
f1_score          0.110556
frequency_true    0.299575
frequency_pred    0.266000
dtype: float64

In [5]:
results = []

for model in ("fedlr", "fedfor"):

    for n_clients in (10, 20, 30, 40, 50, 60, 70, 80, 90, 100):

        dfs = {}

        for dataset in dataset_names.keys():

            dft = pd.read_csv(
                f"history/{n_clients}_clients_{dataset}_{model}.csv"
            )

            df = pd.DataFrame()
        
            # get percentiles (0, 10, 25, 50, 75, 90, 100)
            df = dft[[f'Client {i+1} Accuracy' for i in range(n_clients)]].T.describe(
                percentiles=[0.1, 0.25, 0.5, 0.75, 0.9]
                ).loc[['min', '10%', '25%', '50%', '75%', '90%', 'max']].T

            df['n_clients'] = n_clients

            df['global'] = dft['Global Accuracy']

            dfs[dataset] = df

        for comm_cost in (0.0, 0.0125, 0.025, 0.05, 0.1):
            for dataset in dataset_names.keys():

                dfs[dataset]['ground_truth'] = dfs[dataset].apply(
                    lambda row: 1*(row['global'] > row['max'] + comm_cost), axis=1
                )

                X = dfs[dataset].drop(columns=['ground_truth', 'global'])
                y = dfs[dataset]['ground_truth']

                nsplit = int(len(X) / 2)

                X_train = X.iloc[:nsplit, :].reset_index(drop=True)
                y_train = y.iloc[:nsplit].reset_index(drop=True)
                X_test = X.iloc[nsplit:, :].reset_index(drop=True)
                y_test = y.iloc[nsplit:].reset_index(drop=True)

                # for dataset in dataset_names.keys():

                #     dfs[dataset]['ground_truth'] = dfs[dataset].apply(
                #         lambda row: 1*(row['global'] > row['max'] + comm_cost), axis=1
                #     )
                #     if dataset != target_dataset:
                #         X_train = pd.concat([X_train, dfs[dataset].drop(columns=['ground_truth', 'global'])], ignore_index=True)
                #         y_train = pd.concat([y_train, dfs[dataset]['ground_truth']], ignore_index=True)

                #     else:
                #         X_test = dfs[dataset].drop(columns=['ground_truth', 'global'])
                #         y_test = dfs[dataset]['ground_truth']

                if len(y_train.unique()) < 2:
                    if y_train.unique()[0] == 0:
                        y_pred = np.zeros_like(y_test)
                    else:
                        y_pred = np.ones_like(y_test)
                else:
                    # est = LogisticRegression(max_iter=1000, random_state=42)
                    est = RandomForestClassifier(n_estimators=100, random_state=42)

                    est.fit(X_train, y_train)
                    y_pred = est.predict(X_test)

                results.append({
                    "model": model,
                    "n_clients": n_clients,
                    "comm_cost": comm_cost,
                    "target_dataset": dataset,
                    "accuracy": accuracy_score(y_test, y_pred),
                    "precision": precision_score(y_test, y_pred, zero_division=0),
                    "recall": recall_score(y_test, y_pred, zero_division=0),
                    "f1_score": f1_score(y_test, y_pred, zero_division=0),
                    "frequency_true": np.mean(y_test),
                    "frequency_pred": np.mean(y_pred),
                })

                print(f"Completed: model={model}, n_clients={n_clients}, comm_cost={comm_cost}, target_dataset={target_dataset}")
                print(f"  Accuracy: {results[-1]['accuracy']:.4f}, Precision: {results[-1]['precision']:.4f}, Recall: {results[-1]['recall']:.4f}, F1-score: {results[-1]['f1_score']:.4f}")
                print(f"Frequency True: {results[-1]['frequency_true']:.4f}, Frequency Pred: {results[-1]['frequency_pred']:.4f}")
                results_df = pd.DataFrame(results)
                results_df.to_csv("final/split_estimator_results.csv", index=False)

                


Completed: model=fedlr, n_clients=10, comm_cost=0.0, target_dataset=spambase
  Accuracy: 1.0000, Precision: 0.0000, Recall: 0.0000, F1-score: 0.0000
Frequency True: 0.0000, Frequency Pred: 0.0000
Completed: model=fedlr, n_clients=10, comm_cost=0.0, target_dataset=spambase
  Accuracy: 0.8300, Precision: 0.8723, Recall: 0.9425, F1-score: 0.9061
Frequency True: 0.8700, Frequency Pred: 0.9400
Completed: model=fedlr, n_clients=10, comm_cost=0.0, target_dataset=spambase
  Accuracy: 0.9900, Precision: 0.0000, Recall: 0.0000, F1-score: 0.0000
Frequency True: 0.0100, Frequency Pred: 0.0000
Completed: model=fedlr, n_clients=10, comm_cost=0.0, target_dataset=spambase
  Accuracy: 1.0000, Precision: 1.0000, Recall: 1.0000, F1-score: 1.0000
Frequency True: 1.0000, Frequency Pred: 1.0000
Completed: model=fedlr, n_clients=10, comm_cost=0.0125, target_dataset=spambase
  Accuracy: 1.0000, Precision: 0.0000, Recall: 0.0000, F1-score: 0.0000
Frequency True: 0.0000, Frequency Pred: 0.0000
Completed: model=

In [6]:
results_df[['accuracy', 'precision', 'recall', 'f1_score', 'frequency_true', 'frequency_pred']].mean()

accuracy          0.957725
precision         0.363257
recall            0.340860
f1_score          0.342446
frequency_true    0.299750
frequency_pred    0.300675
dtype: float64

In [7]:
# comm_cost = 0
# model = "fedfor"
# target_dataset = "adult"
# n_clients = 10

In [8]:

# dfs = {}
# for dataset in dataset_names.keys():

#     filepath = f"history/{n_clients}_clients_{dataset}_{model}.csv"

#     dft = pd.read_csv(filepath)

#     df = pd.DataFrame()
#         # 
#     # get percentiles (0, 10, 25, 50, 75, 90, 100)
#     df = dft[[f'Client {i+1} Accuracy' for i in range(n_clients)]].T.describe(
#         percentiles=[0.1, 0.25, 0.5, 0.75, 0.9]
#         ).loc[['min', '10%', '25%', '50%', '75%', '90%', 'max']].T

#     df['global'] = dft['Global Accuracy']
#     df['ground_truth'] = df.apply(lambda row: 1*(row['global'] > row['max']), axis=1)

#     dfs[dataset] = df

In [9]:
# dfs[target_dataset]

In [10]:
# for dataset in dataset_names.keys():

#     print(f"Dataset: {dataset_names[dataset]}")
#     print(dfs[dataset]['ground_truth'].value_counts().sort_index())
#     print()

In [11]:
# # combine dataset in datasets != target_dataset in X (features != global accuracy) and y (target = 1 if global accuracy > max; 0 else 0)

# X_train = pd.DataFrame()
# y_train = pd.Series(dtype=int)

# for dataset in dataset_names.keys():

#     X_ = dfs[dataset].drop(columns=['global'])
#     y_ = (dfs[dataset]['global'] > dfs[dataset]['max']).astype(int)

#     if dataset != target_dataset:
#         X_train = pd.concat([X_train, X_], ignore_index=True)
#         y_train = pd.concat([y_train, y_], ignore_index=True)

#     else:
#         X_test = X_
#         y_test = y_

In [12]:
# print(f"Train samples: {X_train.shape[0]}, Test samples: {X_test.shape[0]}")

In [13]:
# est = RandomForestClassifier(n_estimators=10, random_state=42)

# # X_test = X_train
# # y_test = y_train

# est.fit(X_train, y_train)
# y_pred = est.predict(X_test)

In [14]:
# print((y_pred == y_test).mean())

In [15]:
# from sklearn.metrics import classification_report

# print(classification_report(y_test, y_pred, zero_division=0))

In [16]:
# y_test.value_counts()

In [17]:
# pd.Series(y_pred).value_counts()